In [ ]:
import keras 
from keras.datasets import mnist
from keras import backend as K 

In [2]:
(x_train, y_train),(x_test,y_test) = mnist.load_data()

In [6]:
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)
input_shape = (28, 28, 1)

In [7]:
num_classes=10
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [14]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [15]:
#PREPROCESSING THE DATA

In [ ]:
#CREATING THE CONVOLUTION NEURAL NETWORK

In [17]:
import tensorflow as tf

In [18]:
cnn = tf.keras.models.Sequential()

In [19]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=(3,3), activation='relu',input_shape=(28,28,1)))
cnn.add(tf.keras.layers.Conv2D(filters=64, kernel_size=(3,3), activation='relu'))

In [20]:
cnn.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2),strides=2))

In [21]:
cnn.add(tf.keras.layers.Dropout(0.25))

In [22]:
cnn.add(tf.keras.layers.Flatten())

In [23]:
cnn.add(tf.keras.layers.Dense(units=256,activation='relu'))

In [24]:
cnn.add(tf.keras.layers.Dropout(0.5))

In [25]:
cnn.add(tf.keras.layers.Dense(units=10,activation='softmax'))

In [26]:
cnn.compile(loss=keras.losses.categorical_crossentropy,optimizer=keras.optimizers.Adadelta(),metrics=['accuracy'])

In [28]:
hist = cnn.fit(x_train, y_train,batch_size=128,epochs=10,verbose=1,validation_data=(x_test, y_test))
print("The model has successfully trained")

Epoch 1/10
469/469 [==============================] - 78s 167ms/step - loss: 2.3026 - accuracy: 0.1119 - val_loss: 2.3026 - val_accuracy: 0.1135
Epoch 2/10
469/469 [==============================] - 77s 165ms/step - loss: 2.3026 - accuracy: 0.1124 - val_loss: 2.3025 - val_accuracy: 0.1135
Epoch 3/10
469/469 [==============================] - 77s 165ms/step - loss: 2.3025 - accuracy: 0.1124 - val_loss: 2.3025 - val_accuracy: 0.1135
Epoch 4/10
469/469 [==============================] - 77s 165ms/step - loss: 2.3025 - accuracy: 0.1124 - val_loss: 2.3025 - val_accuracy: 0.1135
Epoch 5/10
469/469 [==============================] - 78s 165ms/step - loss: 2.3025 - accuracy: 0.1124 - val_loss: 2.3025 - val_accuracy: 0.1135
Epoch 6/10
469/469 [==============================] - 77s 164ms/step - loss: 2.3025 - accuracy: 0.1124 - val_loss: 2.3025 - val_accuracy: 0.1135
Epoch 7/10
469/469 [==============================] - 77s 165ms/step - loss: 2.3025 - accuracy: 0.1124 - val_loss: 2.3024 - val_ac

In [30]:
cnn.save('mnist.h5')
print("Saving the model as mnist.h5")

Saving the model as mnist.h5


In [31]:
#CREATING THE GUI FOR TO PREDICT THE DIGITS!!

In [34]:
from keras.models import load_model
from tkinter import *
import tkinter as tk
import win32gui
from PIL import ImageGrab, Image
import numpy as np

In [35]:
model = load_model('mnist.h5')

In [37]:
#CRETING A FUNCTION TO PREDICT THE NUMBER!!!!

In [38]:
def predict(img):
    #RESIZING THE IMAGE 
    img = img.resize((28,28))
    #CONVERTING THE IMAGE TO GRAYSCALE
    img = img.convert('L')
    #CONVERTING THE IMAGE INTO AN ARRAY
    img = np.array(img)
    img = img.reshape(1,28,28,1)
    img = img/255.0
    res = model.predict([img])[0]
    return np.argmax(res), max(res)

In [40]:
#CREATONG A CLASS FOR THE TKINTER INTERFACE 

In [44]:
class App(tk.Tk):
    def __init__(self):
        tk.Tk.__init__(self)
        self.x = self.y = 0
        # Creating elements
        self.canvas = tk.Canvas(self, width=300, height=300, bg = "white", cursor="cross")
        self.label = tk.Label(self, text="Thinking..", font=("Helvetica", 48))
        self.classify_btn = tk.Button(self, text = "Recognise", command =         self.classify_handwriting) 
        self.button_clear = tk.Button(self, text = "Clear", command = self.clear_all)
        # Grid structure
        self.canvas.grid(row=0, column=0, pady=2, sticky=W, )
        self.label.grid(row=0, column=1,pady=2, padx=2)
        self.classify_btn.grid(row=1, column=1, pady=2, padx=2)
        self.button_clear.grid(row=1, column=0, pady=2)
        #self.canvas.bind("<Motion>", self.start_pos)
        self.canvas.bind("<B1-Motion>", self.draw_lines)
    def clear_all(self):
        self.canvas.delete("all")
    def classify_handwriting(self):
        HWND = self.canvas.winfo_id() # get the handle of the canvas
        rect = win32gui.GetWindowRect(HWND) # get the coordinate of the canvas
        im = ImageGrab.grab(rect)
        digit, acc = predict(im)
        self.label.configure(text= str(digit)+', '+ str(int(acc*100))+'%')
    def draw_lines(self, event):
        self.x = event.x
        self.y = event.y
        r=8
        self.canvas.create_oval(self.x-r, self.y-r, self.x + r, self.y + r, fill='black')
app = App()
mainloop()